# Import Data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import norm
import scipy.stats as stats
import pylab  

import warnings
warnings.filterwarnings('ignore')

import haversine as hs

In [ ]:
fares = pd.read_csv('trip_fare.csv')
trips = pd.read_csv('trip_data.csv')
df = fares.merge(trips)

In [ ]:
df.to_pickle('nyc_taxi_original.pickle')

In [ ]:
df_orig = pd.read_pickle('nyc_taxi_original.pickle')
df_orig.columns = [i.strip() for i in list(df_orig.columns)]

In [ ]:
#print the number of rows and columns of the loaded dataframe
print('The dataset has',df.shape[0],'rows and',df.shape[1],'columns.')

# Data Cleaning and Feature Prep

In [ ]:
# correct variable names for later convenience
df.columns = [i.strip() for i in list(df.columns)]
df.info() # take a look at current dataframe information

In [ ]:
#convert second duration 
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime']=pd.to_datetime(df['dropoff_datetime'])

In [ ]:
#convert date string to datetime
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime']=pd.to_datetime(df['dropoff_datetime'])

In [ ]:
# extract pickup and dropoff travel time in hours
df['trip_duration_hour']=df['trip_time_in_secs']/3600 

# extract pickup and dropoff hour
df['dropoff_hour']=df['dropoff_datetime'].dt.hour
df['pickup_hour']=df['dropoff_datetime'].dt.hour

# extract pickup and dropoff day
df['pickup_day_date']=df['pickup_datetime'].dt.day
df['dropoff_day_date']=df['dropoff_datetime'].dt.day

# extract pickup and dropoff day of week
df['pickup_day']=df['pickup_datetime'].dt.day_name()
df['dropoff_day']=df['dropoff_datetime'].dt.day_name()

# create a log transformed distance
df['trip_logdist'] = df['trip_distance'].apply(lambda x: np.log(x + .01))

# create geographic distance
df['trip_geodistance'] = df.apply(lambda x: geodistance(x['pickup_latitude'], x['pickup_longitude'], 
                                              x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

# calculate speed
df['trip_speed'] = df['trip_distance']/df['trip_duration_hour'] 

In [ ]:
def get_wknd_wkdy(x): 
    
    wknd = ['Saturday', 'Sunday']
    wkdy = ['Monday', 'Tuesday', 'Wednesday', 'Thursday' ,'Friday']
    
    if x in wknd:
        
        return 'Weekend'
    
    else: 
        
        return 'Weekday'

In [ ]:
df['pickup_day_type'] = df['pickup_day'].apply(lambda x:get_wknd_wkdy(x))
df['dropoff_day_type'] = df['dropoff_day'].apply(lambda x:get_wknd_wkdy(x))

In [ ]:
def geodistance(x_long, x_lat, y_long, y_lat): 
    
    loc_1 = (x_lat, x_long)
    loc_2 = (y_lat, y_long)
    geodist = hs.haversine(loc_1,loc_2)
    hs.haversine(loc_1,loc_2,unit='mi')
    
    return geodist

In [ ]:
nyc_boroughs={
    'manhattan':{
        'min_lng':-74.0479,
        'min_lat':40.6829,
        'max_lng':-73.9067,
        'max_lat':40.8820
    },
    
    'queens':{
        'min_lng':-73.9630,
        'min_lat':40.5431,
        'max_lng':-73.7004,
        'max_lat':40.8007

    },

    'brooklyn':{
        'min_lng':-74.0421,
        'min_lat':40.5707,
        'max_lng':-73.8334,
        'max_lat':40.7395
        
   },

    'bronx':{
        'min_lng':-73.9339,
        'min_lat':40.7855,
        'max_lng':-73.7654,
        'max_lat':40.9176

    },

    'staten_island':{
        'min_lng':-74.2558,
        'min_lat':40.4960,
        'max_lng':-74.0522,
        'max_lat':40.6490

    }
    
    
    
}

In [ ]:
nyc_airports={  
    
    
    'JFK_Airport' :{
       'min_lng':-73.8352,
       'min_lat':40.6195,
       'max_lng':-73.7401, 
       'max_lat':40.6659

    },
    
    'Newark_Airport':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

    },
    
    'LaGuardia_Airport':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
    
    }
    
}    
                         

In [ ]:
def getBorough(lat,lng):
    
    locs=nyc_boroughs.keys()
    for loc in locs:
        if lat>=nyc_boroughs[loc]['min_lat'] and lat<=nyc_boroughs[loc]['max_lat'] and lng>=nyc_boroughs[loc]['min_lng'] and lng<=nyc_boroughs[loc]['max_lng']:
            return loc
    return 'others'

In [ ]:
def getAirport(lat,lng):
    
    locs=nyc_airports.keys()
    for loc in locs:
        if lat>=nyc_airports[loc]['min_lat'] and lat<=nyc_airports[loc]['max_lat'] and lng>=nyc_airports[loc]['min_lng'] and lng<=nyc_airports[loc]['max_lng']:
            return loc
    return 'not_airport'

In [ ]:
df['pickup_borough']=df.apply(lambda row:getBorough(row['pickup_latitude'],row['pickup_longitude']),axis=1)
df['dropoff_borough']=df.apply(lambda row:getBorough(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [ ]:
df['pickup_airport']=df.apply(lambda row:getAirport(row['pickup_latitude'],row['pickup_longitude']),axis=1)
df['dropoff_airport']=df.apply(lambda row:getAirport(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [ ]:
def origin_dest(borough, airport): 
    
    if airport != 'not_airport':
        
        return airport
        
    return borough

In [ ]:
df['pickup_origin'] = df.apply(lambda row:origin_dest(row['pickup_borough'],row['pickup_airport']),axis=1)

In [ ]:
df['dropoff_dest'] = df.apply(lambda row:origin_dest(row['dropoff_borough'],row['dropoff_airport']),axis=1)

In [ ]:
df.to_pickle('nyc_taxi_processed.pickle')

<b>2. Records that met the following criteria were excluded from all subsequent analyses:</b>

Distance of 0 mile or distance ≥ 50 miles;
Duration of 0 minute or duration ≥ 200 minutes;

Average speed ≤ 1 MPH or average speed ≥ 240 MPH;

Base fare < 2.50 or ≥ 250.00, or tip amount > twice the base fare;

With invalid longitude or latitude data, or with a trip distance shorter than the geographic distance between pickup and drop-off point by more than 1 mile, or traveled outside the 5 boroughs or 3 airports of NYC.

In [ ]:
# To remove trip with extreme distances: 
## a. for trips with distance of 0 mile: keep trips with different pickup/dropoff locations
filter1 = (df['trip_distance']==0) & (df['pickup_borough']!=df['dropoff_borough'])
## b. for trips with distance >0 mile: keep trip within 50 miles (99.99% percentile was 41.25 miles). 
filter2 = (df['trip_distance']>0) & (df['trip_distance']<50) 

# apply criteria
start_n = len(df)
df = df [filter1 | filter2]
print('Step 1 excludes',start_n - len(df),'records, current sample size=',len(df))

# To remove trips with extreme long duration:
## a. for trips with duration of 0 minute: keep trips with different pickup/dropoff locations
filter3 = (df['trip_duration_hour']==0) & (df['pickup_borough']!=df['dropoff_borough'])

## b. for trips with distance >0 min: keep trips under 200 mins (99% percentile was 62 mins) which is 3.33 hours
filter4 = (df['trip_duration_hour']>0) & (df['trip_duration_hour'] <3.33) 

# apply criteria
start_n = len(df)
df = df [filter3 | filter4]
print('Step 2 excludes',start_n - len(df),'records, current sample size=',len(df))

# To remove trips with extreme fast speed
## a. keep trip with average speed <= 240 MPH (i.e. 240 MPH which is unlikely)
## b. keep trip with average speed >= 1 MPH (i.e. 1 MPH which is unlikely)
filter5 = ((df['trip_speed']< 240) & (df['trip_speed'] > 1))|(df['trip_distance']==0) | (df['trip_duration_hour'] ==0) 

# apply criteria
start_n = len(df)
df = df [filter5]
print('Step 3 excludes',start_n - len(df),'records, current sample size=',len(df))

# To remove trips with extreme fare amounts:
## a. keep trips with fare >= the minimum amount $2.5
## b. keep fare amount < $250 (99.99% percentile was $200)
filter6 = (df['fare_amount']>=2.5) & (df['fare_amount'] < 250)
## c. keep trips with tip amount <= twice the fare amount (implausible values)
filter7 = df['tip_amount'] <= 2*df['fare_amount']

# apply criteria
start_n = len(df)
df = df [filter6 & filter7]
print('Step 4 excludes',start_n - len(df),'records, current sample size=',len(df))

# To remove trips with invalid geographic location
## a. Remove trips with invalid GPS data
filter8 = (df['pickup_longitude']!=0) & (df['pickup_latitude']!=0) & (df['dropoff_longitude']!=0) & (df['dropoff_latitude']!=0)

## b. Keep trips with travel distance >= GPS distance - 1 mile (implausible value)
filter9 = df['trip_distance'] >= (df['trip_geodistance'] -1)

## apply criteria
start_n = len(df)
df = df[filter8 & filter9]
print('Step 5 excludes',start_n - len(df),'records, current sample size=',len(df))

# c. Remove trips out of new york city area (5 boroughs plus 3 airports)
filter10 = (df['rate_code'] < 4)

# c. Remove trips without cash or credit card payment
filter11 = (df['payment_type'] == 'CSH') | (df['payment_type'] == 'CRD')

## apply criteria
start_n = len(df)
df = df[filter10 & filter11]
print('Step 6 excludes',start_n - len(df),'records, current sample size=',len(df))

In [ ]:
# Impute distance, time and speed data

# calculate median speed for valid trips
median_speed = df['trip_speed'].loc[(df['trip_distance']>0) & (df['trip_duration_hour']>0)].median()

# index type of data error
dist_0 = (df['trip_distance']==0) & (df['trip_duration_hour']>0)
time_0 = (df['trip_distance']>0) & (df['trip_duration_hour']==0)
ratecode_2 = (df['trip_distance']==0) & (df['trip_duration_hour']==0) & (df['rate_code']==2)
other_err = (df['trip_distance']==0) & (df['trip_duration_hour']==0) & (df['rate_code']!=2)

# For trips with distance = 0 and time >0, replaced by median speed * trip time                                  
df['trip_distance'].loc[dist_0] = df['trip_duration_hour'].loc[dist_0]*median_speed

# For trips with distance > 0 and time = 0, replaced by trip distance /median speed 
df['trip_duration_hour'].loc[time_0] = df['trip_distance'].loc[time_0]/median_speed

# For trips with distance ==0 and time = 0 and rate code ==2, replaced by median distance and time for rate code ==2
df['trip_duration_hour'].loc[ratecode_2] = df['trip_duration_hour'].loc[(df['trip_distance']>0) & (df['trip_duration_hour']>0) \
                                                      & (df['rate_code']==2)].median()
df['trip_distance'].loc[ratecode_2] = df['trip_distance'].loc[(df['trip_distance']>0) & (df['trip_duration_hour']>0) \
                                                      & (df['rate_code']==2)].median()

# For trips with distance ==0 and time = 0 and rate code !=2, replaced using median distance or time/dollar*fare
time_p_usd = df['trip_duration_hour']/df['fare_amount']
distance_p_usd = df['trip_distance']/df['fare_amount']

df['trip_duration_hour'].loc[other_err] = df['fare_amount'].loc[other_err] * time_p_usd.median()
df['trip_distance'].loc[other_err] = df['fare_amount'].loc[other_err] * distance_p_usd.median()

# recalculate speed after impution
df['trip_speed'] = df['trip_distance']/df['trip_duration_hour']

df.to_pickle('nyc_taxi_prepped.pickle')

df = pd.read_pickle('nyc_taxi_prepped.pickle')

In [ ]:
# plot distance with and without data cleaning
fig, axs = plt.subplots(1,2)
fig.set_size_inches(9, 3)
fig.subplots_adjust(wspace=.5)

# We can set the number of bins with the `bins` kwarg
#axs[0].hist(df_orig['trip_distance'],bins=100);
#axs[0].set_title('Histogram of distance, Raw Data')
#axs[0].set_xlabel('Distance (mile)')
#axs[0].set_ylabel('Number of records')

axs[1].hist(df['pickup_hour'],bins=100);
axs[1].set_title('Histogram of Pickup Hour, Cleaned data')
axs[1].set_xlabel('Pick Up Hour')
axs[1].set_ylabel('Number of records');

#fig.savefig('fig2.svg', format='svg',dpi=400, bbox_inches='tight')

#print('The average distance is',round(df['trip_distance'].mean(),1),', the standard deviation is',round(df['trip_distance'].std(),1))
#print('The median distance is',round(df['trip_distance'].median(),1),', the 95% of trips distance fell between',
#round(df['trip_distance'].quantile(.025),2),'to',round(df['trip_distance'].quantile(.975), 1))

In [ ]:
# plot distance with and without data cleaning
fig, axs = plt.subplots(1,2)
fig.set_size_inches(9, 3)
fig.subplots_adjust(wspace=.5)

# We can set the number of bins with the `bins` kwarg
axs[0].hist(df_orig['trip_distance'],bins=100);
axs[0].set_title('Histogram of distance, Raw Data')
axs[0].set_xlabel('Distance (mile)')
axs[0].set_ylabel('Number of records')

axs[1].hist(df['trip_distance'],bins=100);
axs[1].set_title('Histogram of distance, Cleaned data')
axs[1].set_xlabel('Distance (mile)')
axs[1].set_ylabel('Number of records');

#fig.savefig('fig2.svg', format='svg',dpi=400, bbox_inches='tight')

print('The average distance is',round(df['trip_distance'].mean(),1),', the standard deviation is',round(df['trip_distance'].std(),1))
print('The median distance is',round(df['trip_distance'].median(),1),', the 95% of trips distance fell between',
round(df['trip_distance'].quantile(.025),2),'to',round(df['trip_distance'].quantile(.975), 1))

In [ ]:
# create a log transformed distance
df['trip_logdist'] = df['trip_distance'].apply(lambda x: np.log(x))

# plot log-transformed distance distribution and QQ plot of log-transformed distance
fig, axs = plt.subplots(1,2)
fig.set_size_inches(9, 3)
fig.subplots_adjust(wspace=.5)

# We can set the number of bins with the `bins` kwarg
axs[0].hist(df['trip_logdist'], bins=50);
axs[0].set_title('A. Histogram of log-transformed distance')
axs[0].set_ylabel('Number of records');

stats.probplot(df['trip_logdist'].sample(3000), dist="norm", plot=axs[1])# sample data points to reduce figure size
axs[1].set_title('B. Q-Q plot of log-transformed distance');
#fig.savefig('fig3.svg', format='svg',bbox_inches='tight')

In [ ]:
# Calculate mean and median trip distance by grouping pick up hours of day
df_hourly_distance=df[['trip_distance','pickup_hour']].groupby(by='pickup_hour').mean()
df_hourly_distance.columns=['mean_trip_distance']
df_hourly_distance['median_trip_distance']=df[['trip_distance','pickup_hour']].groupby(by='pickup_hour').median()
#df_hourly_distance.reset_index().to_csv('test.csv')

In [ ]:
sns.lineplot(data=df_hourly_distance)

Mean distances were constantly higher than median distance during each hour. 
This is most likely because the distribution of trip distance is right-skewed, and the mean could be inflated by a few long-distance trips. As such, the median is a fairer representation of the central tendency of all distances.